In [1]:
!pip install  langchain

In [2]:
!pip install  langchain-google-genai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 28.3 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.17 which is incompatible.


In [3]:
!pip install langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.49
    Uninstalling langchain-core-0.3.49:
      Successfully uninstalled langchain-core-0.3.49
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.7
    Uninstalling langchain-text-splitters-0.3.7:
      Successfully uninstalled langchain-text-splitters-0.3.7
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.22
    Uninstalling langchain-0.3.22:
      Successfully uninstalled langchain-0.3.22


In [5]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.0 MB/s eta 0:00:00


In [6]:
# Step 1: Import Required Libraries
import streamlit as st
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory


In [7]:
# Step 2: Load Gemini API Key from File (DO NOT hardcode keys)
with open("gemini.txt", "r") as f:
    GOOGLE_API_KEY = f.read().strip()


In [9]:
# Step 3: Initialize the Google Gemini Chat Model
chat_model = ChatGoogleGenerativeAI(
    api_key=GOOGLE_API_KEY,
    model="gemini-1.5-pro",
    temperature=0.7
)


In [10]:
# Step 4: Define Chat History Storage using SQLite
def get_session_message_history_from_db(session_id):
    return SQLChatMessageHistory(
        session_id=session_id,
        connection="sqlite:///sqlite.db"
    )


In [11]:
# Step 5: Define the System Prompt Template for the Chatbot
chat_template = ChatPromptTemplate(
    messages=[
        ("system", """
        You are a helpful, polite AI assistant that only answers Data Science-related questions.
        Politely decline any non-related queries and encourage the user to ask something about Data Science.
        Give short, precise, and informative answers.
        """),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{human_input}")
    ]
)


In [12]:
# Step 6: Define the Output Parser
output_parser = StrOutputParser()


In [13]:
# Step 7: Set Up the Complete Conversation Chain (with memory)
conversation_chain = RunnableWithMessageHistory(
    chat_template | chat_model | output_parser,
    get_session_message_history_from_db,
    input_messages_key="human_input",
    history_messages_key="history"
)


In [15]:
%%writefile app.py
import streamlit as st
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory

# ✅ DO: Always store API keys securely
with open("gemini.txt", "r") as f:
    GOOGLE_API_KEY = f.read().strip()

# ✅ Initialize Chat Model (Generative AI Model)
chat_model = ChatGoogleGenerativeAI(
    api_key=GOOGLE_API_KEY,
    model="gemini-1.5-pro",
    temperature=0.7  # ❗ DO: Use lower temperature for more factual responses
)

# ✅ Data Structure: SQLite-based Message History (acts like a persistent dictionary)
def get_session_message_history_from_db(session_id):
    return SQLChatMessageHistory(session_id=session_id, connection="sqlite:///sqlite.db")

# ✅ Define Static FAQ-like Prompt Template
chat_template = ChatPromptTemplate(
    messages=[
        ("system", """
        You are a helpful and polite AI assistant trained to answer only data science-related queries.
        If a user asks about non-related topics, kindly redirect them back to data science.
        Your responses should be short, accurate, and educational.
        """),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{human_input}")
    ]
)

# ✅ Output Handler
output_parser = StrOutputParser()

# ✅ Conversation Handler with History
conversation_chain = RunnableWithMessageHistory(
    chat_template | chat_model | output_parser,
    get_session_message_history_from_db,
    input_messages_key="human_input",
    history_messages_key="history"
)

# ✅ UI
st.title("📘 Simple FAQs Data Science Chatbot")
st.markdown("Ask me any **Data Science** FAQ (e.g., What is overfitting? What is linear regression?)")

# ✅ DO: Maintain session state for persistent conversations
if "session_id" not in st.session_state:
    st.session_state.session_id = "user_1"  # DO: Use unique ID in production
if "messages" not in st.session_state:
    st.session_state.messages = []  # ✅ List to track messages

# ✅ Display past messages
for msg in st.session_state.messages:
    with st.chat_message(msg["role"]):
        st.markdown(msg["content"])

# ✅ Input Area
user_input = st.chat_input("Ask a Data Science question...")
if user_input:
    # ✅ DO: Validate the input (basic form here)
    if len(user_input.strip()) == 0:
        st.warning("Please enter a valid question.")
    else:
        st.session_state.messages.append({"role": "user", "content": user_input})
        with st.chat_message("user"):
            st.markdown(user_input)

        # Get AI response
        config = {"configurable": {"session_id": st.session_state.session_id}}
        response = conversation_chain.invoke({"human_input": user_input}, config=config)

        # Show response
        st.session_state.messages.append({"role": "assistant", "content": response})
        with st.chat_message("assistant"):
            st.markdown(response)


Writing app.py
